Author: Rashad Haddad  
Description: Compute scores for example JSONs.  Data is relevant as of Mar 3rd, 2023

In [1]:
import pandas as pd
import numpy as np

import json
import sys
if "../" not in sys.path:
    sys.path.append("../")
from lib.compute_score import compute_score

In [2]:
def make_report(dat):
    # instantiate obligor
    obl = compute_score(input_data=dat, start_alpha=10, start_beta=10, protocol_name="aave_v3")
    
    # get counts of each transaction type for reference...
    counts = {'borrow': 0, 'repay': 0, 'liquidation': 0, 'deposit': 0, 'withdraw': 0}
    min_timestamp = np.inf
    max_timestamp = 0
    for tx in dat:
        counts[tx['type']] = counts.get(tx['type'],0) + 1

        if int(tx['timestamp']) > max_timestamp:
            max_timestamp = int(tx["timestamp"])
        if int(tx['timestamp']) < min_timestamp:
            min_timestamp = int(tx["timestamp"])

    # get report
    print("Number of transactions --> ", len(dat))
    print("Number of borrows --> ", counts['borrow'])
    print("Number of repays --> ", counts['repay'])
    print("Number of liquidates --> ", counts['liquidation'])
    print("Number of deposits --> ", counts['deposit'])
    print("Number of withdraws --> ", counts['withdraw'])
    print("Finishing alpha (good credit param) --> ", obl._alpha)
    print("Finishing beta (bad credit param) --> ", obl._beta)
    print("Finishing score --> ", obl.get_score())
    print("Finishing 95% confidence interval --> ", obl.get_conf_interval())
    print("\nmin timestamp --> ", min_timestamp)
    print("\nmax timestamp --> ", max_timestamp)

    return obl


### Example 1 0x2d3178af3dfbb679716cc14e245be0a9e5945500

In [3]:
obl1 = make_report(json.load(open("../../../testData/0x2d31-49-32-65.json","r")))

Number of transactions -->  52
Number of borrows -->  20
Number of repays -->  5
Number of liquidates -->  0
Number of deposits -->  25
Number of withdraws -->  2
Finishing alpha (good credit param) -->  17.39337698223064
Finishing beta (bad credit param) -->  18.30814684632559
Finishing score -->  49
Finishing 95% confidence interval -->  (32, 65)

min timestamp -->  1674860423

max timestamp -->  1677831467


Example 1 ties with ts...

### Example 2 0x9600a48ed0f931d0c422d574e3275a90d8b22745

In [5]:
obl2 = make_report(json.load(open("../../../testData/0x9600-67-57-76.json","r")))

Number of transactions -->  259
Number of borrows -->  85
Number of repays -->  24
Number of liquidates -->  0
Number of deposits -->  100
Number of withdraws -->  50
Finishing alpha (good credit param) -->  65.75930237060297
Finishing beta (bad credit param) -->  32.784419407149244
Finishing score -->  67
Finishing 95% confidence interval -->  (57, 76)

min timestamp -->  1676169899

max timestamp -->  1677880187


ties to TS

With a lower bound at 61, we are very confident this is a reliable borrower. This borrower can safely be given high efficiency terms.

### Example 3 0xa3fa88658d7f6ceea0288033e85de77d3c02f779 

In [6]:
obl3 = make_report(json.load(open("../../../testData/0xa3fa-57-38-77.json","r")))

Number of transactions -->  8
Number of borrows -->  1
Number of repays -->  2
Number of liquidates -->  0
Number of deposits -->  3
Number of withdraws -->  2
Finishing alpha (good credit param) -->  14.001376668246658
Finishing beta (bad credit param) -->  10.487846014458789
Finishing score -->  57
Finishing 95% confidence interval -->  (38, 77)

min timestamp -->  1675891079

max timestamp -->  1677012215


ties to TS.

### Example 4 0x6cfdad0772edc2ea15076a9706ec7c1f0a791ac1

In [8]:
obl4 = make_report(json.load(open("../../../testData/0x6cfd-60-44-76.json")))

Number of transactions -->  38
Number of borrows -->  11
Number of repays -->  9
Number of liquidates -->  0
Number of deposits -->  7
Number of withdraws -->  11
Finishing alpha (good credit param) -->  21.813410432792107
Finishing beta (bad credit param) -->  14.835966627253944
Finishing score -->  60
Finishing 95% confidence interval -->  (44, 76)

min timestamp -->  1675294295

max timestamp -->  1677717719


ties to TS.

## Example 5 

In [10]:
obl5 = make_report(json.load(open("../../../testData/0xbec6-46-28-65.json","r")))

Number of transactions -->  36
Number of borrows -->  13
Number of repays -->  4
Number of liquidates -->  1
Number of deposits -->  14
Number of withdraws -->  4
Finishing alpha (good credit param) -->  13.763601281372745
Finishing beta (bad credit param) -->  16.14149574107211
Finishing score -->  46
Finishing 95% confidence interval -->  (28, 64)

min timestamp -->  1675123559

max timestamp -->  1675995815


ties with TS, weird round on upper bound.

Depsite otherwise good history, we see that given this account has experienced a liquidation, the score isn't that good.

### Example 6 0x23db246031fd6f4e81b0814e9c1dc0901a18da2d

This is an interesting example as there are multiple borrows against the same collat, and a liquidation.

In [13]:
obl6 = make_report(json.load(open("../../../testData/0x23db-46-25-67.json","r")))

Number of transactions -->  5
Number of borrows -->  3
Number of repays -->  0
Number of liquidates -->  1
Number of deposits -->  1
Number of withdraws -->  0
Finishing alpha (good credit param) -->  10
Finishing beta (bad credit param) -->  11.794383676032568
Finishing score -->  46
Finishing 95% confidence interval -->  (25, 67)

min timestamp -->  1674866891

max timestamp -->  1675105907


ties to TS.

Given no repays and a liquidation, a score of 49 is fitting, the range however is very wide here at 29,70, indicating we have a small amount of information on this borrower. A conservative lender should demand high collateral ratios from this borrower.  

### Example 7 0x69bfc96c3983053c36e790c7eb79372bf6979518

In [14]:
obl7 = make_report(json.load(open("../../../testData/0x69bf-48-27-69.json","r")))

Number of transactions -->  3
Number of borrows -->  1
Number of repays -->  0
Number of liquidates -->  1
Number of deposits -->  1
Number of withdraws -->  0
Finishing alpha (good credit param) -->  10
Finishing beta (bad credit param) -->  10.84354189622502
Finishing score -->  48
Finishing 95% confidence interval -->  (27, 69)

min timestamp -->  1676176763

max timestamp -->  1676282087


ties to TS.

The score benefits here (slighly) from the deposit, but the range is wide.

### Example 8 0x2e18e31801870d91114fbfd33c6e4242242bec20

In [15]:
obl8 = make_report(json.load(open("../../../testData/0x23db-46-25-67.json")))

Number of transactions -->  5
Number of borrows -->  3
Number of repays -->  0
Number of liquidates -->  1
Number of deposits -->  1
Number of withdraws -->  0
Finishing alpha (good credit param) -->  10
Finishing beta (bad credit param) -->  11.794383676032568
Finishing score -->  46
Finishing 95% confidence interval -->  (25, 67)

min timestamp -->  1674866891

max timestamp -->  1675105907


ties to TS...

### Example 9  -- 0x2068d14ee50772dd6e89c6f64f32adad0cc936ef

This exampple is interesting due to as of the time it was downloaded there were 19 open positions

In [16]:
obl9 = make_report(json.load(open("../../../testData/0x2068-58-41-74.json")))

Number of transactions -->  20
Number of borrows -->  10
Number of repays -->  0
Number of liquidates -->  0
Number of deposits -->  10
Number of withdraws -->  0
Finishing alpha (good credit param) -->  19.705117750920703
Finishing beta (bad credit param) -->  14.408590330654636
Finishing score -->  58
Finishing 95% confidence interval -->  (41, 74)

min timestamp -->  1677641207

max timestamp -->  1677747995


ties to TS.